In [ ]:
import itertools
import os.path as path

import aesara_theano_fallback.tensor as tt
import astropy.io.fits as fits
import astropy.timeseries as timeseries
import astropy.units as u
import arviz as az
import celerite2
import celerite2.terms as terms
import celerite2.theano.terms as theano_terms
import corner
import exoplanet as xo
import matplotlib
import matplotlib.gridspec as gridspec
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pymc3 as pm
import pymc3_ext as pmx
import scipy.stats as stats

# import third_party.keplersplinev2.keplersplinev2 as ksp

import adhocfitter.astro as astro
import adhocfitter.plotting as plotting
import adhocfitter.priors as priors
import adhocfitter.timeseries as aftimeseries

# Disable OpenMP parallelism
import os
os.environ["OMP_NUM_THREADS"] = "1"

plt.rcParams['font.sans-serif'] = ['TeX Gyre Heros', 'Helvetica', 'Arial', 'sans serif']
plt.rcParams['font.cursive'] = ['TeX Gyre Chorus', 'Apple Chancery', 'cursive']
plt.rcParams['mathtext.fontset'] = 'stixsans'

for p in matplotlib.font_manager.findSystemFonts(fontpaths=[
    '/usr/share/texmf/fonts/opentype/', '/usr/share/fonts/opentype']):
    matplotlib.font_manager.fontManager.addfont(p)

DATA_DIR = '../data'

In [ ]:
test_epoch = np.array([2110.06588, 1855.2442])  # TJD
test_epoch_unc = np.array([0.00028, 0.0022])
test_period = np.array([9.127055, 3.09833])  # day
test_period_unc = np.array([0.0000073, 0.000021])  #np.array([0.00001, 0.000023])
test_last_seen = np.array([2111.0, 1855.3])
# 2073.55739092,  2073.55767828,  2073.55796009]),
#  array([ 1907.91526705,  1907.91617858,  1907.91709297])
test_epoch = (test_last_seen - test_epoch) // test_period * test_period + test_epoch
print(test_epoch)
test_duration = np.array([4.9, 2.0])  # hour
test_duration_unc = np.array([6., 4.])
test_omega_vec = np.array([0., 1.])

test_radius_planet = np.array([0.066, 0.022])  # Stellar radius
test_radius_planet_unc = np.array([0.5, 0.5])
test_impact_param = np.array([0.615, 0.756])
test_limb_dark = np.array([0.37, 0.21])

# Estimated values from previous fits
test_mass_star = 1.086714  # Joint model posterior mean.
test_mass_star_unc = 0.05258207  # Joint model posterior standard deviation.
test_feh = 0.415
test_eep = 380
# A_V extinction
test_av = 0.15  # From Keivan.
test_av_upper = 0.70  # Upper limit from Schlegel 1998 dust map.
# Gaia EDR3 parallax
PARALLAX_GAIA_EDR3 = 5.76044602669829
PARALLAX_GAIA_EDR3_UNC= 0.01048182
# Gaia EDR3 parallax zero-point correction, computed by:
# zpt.get_zpt(
#     phot_g_mean_mag=10.8542,
#     nu_eff_used_in_astrometry=1.5263987,
#     pseudocolour=float('nan'),
#     ecl_lat=-68.38768994372302,
#     astrometric_params_solved=31,  # 5-param solution.
# )
PARALLAX_GAIA_EDR3 -= -0.012446

test_mass_planet = (np.array([75.3, 10.2]) * u.M_earth).to(u.M_sun).value / test_mass_star
test_mass_planet_unc = (np.array([300., 100.]) * u.M_earth).to(u.M_sun).value / test_mass_star # Upper limit.

print(test_mass_planet, test_mass_planet_unc)

# TIC 8.1 values
# test_density_star = 0.839021  # g cm^-3
# test_density_star_unc = 0.190382
# test_radius_star = 1.18093  # R_sun
# test_radius_star_unc = 0.0597739
# test_mass_star = 0.98  # M_sun
# test_mass_star_unc = 0.131459

# FEROS values
# test_density_star = 1.052  # g cm^-3
# test_density_star_unc = 0.052
# test_radius_star = 1.104  # R_sun
# test_radius_star_unc = 0.011
# test_mass_star = 1.001  # M_sun
# test_mass_star_unc = 0.026

# print(test_mass_star/test_radius_star**3 * astro.SOLAR_DENSITY*1e-3)

test_params = {
    "t0": test_epoch,
    "period": test_period,
    "rp": test_radius_planet,
    "b": test_impact_param,
    "ecc_fix": np.array([False, True]),
    "ecc": np.array([0., 0.]),
    "omega": np.array([np.pi/2, np.pi/2]),
    "u": {"tess": test_limb_dark, "Rc": test_limb_dark, "tess_c": test_limb_dark},
#     "rho_star": test_density_star,
    "m_star": test_mass_star,
#     "r_star": test_radius_star,
    "tdur": test_duration,
    "m_planet": test_mass_planet,
    "av": test_av,
    "parallax": PARALLAX_GAIA_EDR3,
    "feh": test_feh,
    "eep": test_eep,
}

prior_unc = {
    "t0": test_epoch_unc,
    "period": test_period_unc,
    "rp": test_radius_planet_unc,
#     "rho_star": test_density_star_unc,
    "m_star": test_mass_star_unc,
#     "r_star": test_radius_star_unc,
    "m_planet": test_mass_planet_unc,
    "tdur": test_duration_unc,
    "av": test_av_upper,
    "parallax": PARALLAX_GAIA_EDR3_UNC,
    "albedo_bond": 0.7,
}

In [ ]:
def add_normal_prior(name, test_params, prior_unc, shape=None):
    if shape is None:
        return pm.Normal(
            name, mu=test_params[name], sd=prior_unc[name])
    else:
        return pm.Normal(
            name, mu=test_params[name], sd=prior_unc[name], shape=shape)

def add_uniform_prior(name, test_params, prior_unc, shape=None):
    lower = test_params[name] - prior_unc[name]
    upper = test_params[name] + prior_unc[name]
    if shape is None:
        return pm.Uniform(
            name, lower=lower, upper=upper, testval=test_params[name])
    else:
        return pm.Uniform(
            name, lower=lower, upper=upper, testval=test_params[name], shape=shape)

In [ ]:
def optimize_model(model, passes=None):
    with model:
        if passes is None:
            return pmx.optimize(start=model.test_point)
        map_soln = pmx.optimize(start=model.test_point, vars=[model[v] for v in passes[0]])
        for p in passes[1:]:
            map_soln = pmx.optimize(start=map_soln, vars=[model[v] for v in p])
        map_soln = pmx.optimize(start=map_soln)
    return map_soln

In [ ]:
rv_table = aftimeseries.read_generic_rv(path.join(DATA_DIR, 'toi_2000_table_08.csv'))
chiron_time, chiron_rv, chiron_rv_unc = aftimeseries.select_rv_by_instrument(rv_table, 'CHIRON')
feros_time, feros_rv, feros_rv_unc = aftimeseries.select_rv_by_instrument(rv_table, 'FEROS')
harps_time, harps_rv, harps_rv_unc = aftimeseries.select_rv_by_instrument(rv_table, 'HARPS')

rv_names = ['chiron', 'feros', 'harps']
rv_times = [chiron_time, feros_time, harps_time]
rv_data = [chiron_rv, feros_rv, harps_rv]
rv_uncs = [chiron_rv_unc, feros_rv_unc, harps_rv_unc]
num_rv_outside = len(rv_data)

rv_times_all = np.concatenate(rv_times)
rv_data_all = np.concatenate(rv_data)
rv_uncs_all = np.concatenate(rv_uncs)

test_gamma = np.array([np.average(i) for i in rv_data])
test_rv_unc = np.array([np.average(i) for i in rv_uncs])
test_jitter = np.array([1e-3]*len(rv_data))

test_params['K'] = np.array([23., 9.])
prior_unc['K'] = np.array([50., 30.])
test_params['rv_gamma'] = test_gamma
prior_unc['rv_gamma'] = np.array([1000.]*num_rv_outside)
test_params['rv_jitter'] = test_jitter
prior_unc['rv_jitter'] = np.array([15., 30., 15.])

test_params['rv_fit_planet'] = np.array([True, True])

test_gamma, test_rv_unc, test_jitter, prior_unc['rv_gamma'], prior_unc['rv_jitter']

In [ ]:
harps_df = pd.read_csv(
    '../data/toi_2000_table_B1.rdb_',
    delim_whitespace=True,
    skiprows=lambda x: x==1,
    comment='#',
)

In [ ]:
activity_index = ['fwhm', 'bis_span', 's_mw', 'rhk']
activity_labels = ['FWHM', 'Bisector span', r'$S_\mathrm{MW}$', r'$R_\mathrm{HK}$']
fig, axs = plt.subplots(
    2, len(activity_index)//2,
    figsize=(8, 2.25*(len(activity_index)//2)),
    dpi=300,
    sharex=True,
)
axs = [axs[0, 0], axs[1, 0], axs[0, 1], axs[1, 1]]
false_alarm_prob = [0.01, 0.001]
for idx, label, ax in zip(activity_index, activity_labels, axs):
    ls_obj = timeseries.LombScargle(harps_df['jdb'], harps_df[idx])
    freq, power = ls_obj.autopower(
        minimum_frequency=1/100.,
        maximum_frequency=1/1.5,
        samples_per_peak=10,
    )
    ax.plot(1/freq, power, zorder=100)
    false_alarm_levels = ls_obj.false_alarm_level(false_alarm_prob, method='bootstrap')
    for fal in false_alarm_levels:
        ax.axhline(fal, linestyle=':', color='gray')
    for p in [90, 17.5]:
        ax.axvline(p, linestyle='--', color='purple')
    ax.set_xscale('log')
    ax.text(0.05, 0.87, label, fontsize='medium', transform=ax.transAxes)
    ax.tick_params(which='both', direction='in', labelsize='small', top=True, right=True)
    ax.set_ylabel('Normalized power')
axs[1].set_xlabel('Period (day)')
ax.set_xlabel('Period (day)')
fig.suptitle('HARPS spectroscopic activity indicators of TOI-2000', fontsize='medium', y=0.95)
fig.tight_layout()
fig.savefig('../plots/toi_2000_harps_rv_activity_periodograms.png', bbox_inches='tight', pad_inches=0.01)
fig.savefig('../plots/toi_2000_harps_rv_activity_periodograms.svg', bbox_inches='tight', pad_inches=0.01)
fig.savefig('../plots/toi_2000_harps_rv_activity_periodograms.pdf', bbox_inches='tight', pad_inches=0.01)

In [ ]:
ls_bis_span = timeseries.LombScargle(
    rv_times_all, np.ones_like(rv_times_all), center_data=False, fit_mean=False)
wn_freq, wn_pow = ls_bis_span.autopower(
    minimum_frequency=1/1000.,
    maximum_frequency=3.,
)
fig, ax = plt.subplots(dpi=144)
ax.plot(1/wn_freq, wn_pow)
ax.set_xscale('log')
ax.set_xlabel('Period (day)')
ax.set_ylabel('Spectral window function')
false_alarm_levels = ls_obj.false_alarm_level(false_alarm_prob)
for fal in false_alarm_levels:
    ax.axhline(fal, linestyle='--', color='gray')
for p in [90, 17.5]:
    ax.axvline(p, linestyle='--', color='gray')

In [ ]:
ls_bis_span = timeseries.LombScargle(np.array(harps_df.jdb), np.array(harps_df.bis_span))
plt.plot(*ls_bis_span.autopower())

In [ ]:
labels = ['1 planet', '2 planets', '3 planets', '4 planets']
plot_peaks = [True] * 3 + [False]
fig, axs = plt.subplots(
    2, 2,
    figsize=(8, 2.25*(len(activity_index)//2)),
    dpi=300,
    sharex=True,
)
axs = [axs[0, 0], axs[1, 0], axs[0, 1], axs[1, 1]]
false_alarm_prob = [0.01, 0.001]
for i, (label, ax, plot_peak) in enumerate(zip(labels, axs, plot_peaks)):
    residual_dir = f'../data/rv_residuals/k{i+1}'
    rv_rjds = []
    rv_residuals = []
    rv_uncs = []
    for fn in [
        'TOI-2000_chiron_DRS-george_Residuals.rdb',
        'TOI-2000_feros_DRS-rafael1_Residuals.rdb',
        'TOI-2000_harps_DRS-armstrong_Residuals.rdb',
    ]:
        residual_df = pd.read_csv(
            path.join(residual_dir, fn),
            delim_whitespace=True,
            skiprows=lambda x: x==1,
            comment='#',
        )
        rv_rjds.append(np.array(residual_df['rjd']))
        rv_residuals.append(np.array(residual_df['res_vrad']))
        rv_uncs.append(np.array(residual_df['svrad']))
    rv_rjd = np.concatenate(rv_rjds)
    rv_sort_idx = np.argsort(rv_rjd)
    rv_residual = np.concatenate(rv_residuals)
    rv_unc = np.concatenate(rv_uncs)
    ls_obj = timeseries.LombScargle(
        t=rv_rjd[rv_sort_idx],
        y=rv_residual[rv_sort_idx],
        dy=rv_unc[rv_sort_idx],
    )
    freq, power = ls_obj.autopower(
        minimum_frequency=1/100.,
        maximum_frequency=1/1.5,
        samples_per_peak=10,
    )
    ax.plot(1/freq, power, zorder=100)
    false_alarm_levels = ls_obj.false_alarm_level(false_alarm_prob, method='bootstrap')
    for fal in false_alarm_levels:
        ax.axhline(fal, linestyle=':', color='gray', zorder=0)
    if plot_peak:
        peak_period = 1. / (freq[np.argmax(power)])
        print(peak_period, '{:.9f}'.format(ls_obj.false_alarm_probability(np.max(power), method='bootstrap')))
        ax.axvline(peak_period, linestyle='--', color='purple')
        # ax.plot(peak_period, 0.6, 'v', color='purple')
        ax.text(0.65, 0.87, f'$P = {peak_period:.2f}$ d', fontsize='medium', transform=ax.transAxes)
    ax.set_xscale('log')
    ax.text(0.25, 0.87, label, fontsize='medium', transform=ax.transAxes)
    ax.tick_params(which='both', direction='in', labelsize='small', top=True, right=True)
    ax.set_ylabel('Normalized power')
    ax.set_xlim(1.3, 130)
    ax.set_ylim(-0.02, 0.62)
axs[1].set_xlabel('Period (day)')
ax.set_xlabel('Period (day)')
fig.suptitle('Periodograms for residuals of Keplerian RV models (TOI-2000)', fontsize='medium', y=0.95)
fig.tight_layout()
fig.savefig('../plots/toi_2000_rv_periodograms.png', bbox_inches='tight', pad_inches=0.01)
fig.savefig('../plots/toi_2000_rv_periodograms.pdf', bbox_inches='tight', pad_inches=0.01)

In [ ]:
'{:.9f}'.format(ls_obj.false_alarm_probability(np.max(power), method='bootstrap'))

In [ ]:
test_params.update({
    'period_rv_only': np.array([89.9, 17.2]),
    't0_rv_only': np.array([2174.56, 2212.0]),
    'K_rv_only': np.array([15.5, 6.]),
})
prior_unc.update({
    'period_rv_only': np.array([20.]),
    'K_rv_only': np.array([30., 15.]),
})
num_planet_rv_only = 2

In [ ]:
def _add_fixed_eccentricity(fix_flag):
    if all(fix_flag):
        return None, [None, None]
    if not any(fix_flag):
        ecc_vec = pmx.UnitDisk(f'sqrt_ecc_vec', testval=np.array([1e-6, 1e-6]), shape=(2, len(fix_flag)))
        ecc = pm.Deterministic('ecc', tt.sum(ecc_vec*ecc_vec, axis=0))
        omega_vec = ecc_vec / tt.sqrt(ecc)
        omega = pm.Deterministic('omega', tt.arctan2(omega_vec[1], omega_vec[0]))
        return ecc, omega_vec
    ecc_stack = []
    omega_vec_stack = []
    for i, ecc_fix in enumerate(fix_flag):
        if ecc_fix:
            ecc_stack.append(tt.as_tensor_variable(0.))
            omega_vec_stack.append(tt.as_tensor_variable(np.array([0., 1.])))
        else:
            ecc_vec = pmx.UnitDisk(f'sqrt_ecc_vec_{i}', testval=np.array([1e-6, 1e-6]))
            ecc = tt.sum(ecc_vec*ecc_vec, axis=0)
            ecc_stack.append(ecc)
            omega_vec_stack.append(ecc_vec / tt.sqrt(ecc))
    ecc = pm.Deterministic('ecc', tt.stack(ecc_stack, axis=0))
    omega_vec = tt.stack(omega_vec_stack, axis=1)
    omega = pm.Deterministic('omega', tt.arctan2(omega_vec[1], omega_vec[0]))
    return ecc, omega_vec

In [ ]:
def make_rv_model_with_rv_only_planets(
    num_planets, ecc_flag, num_planets_rv_only,
    rv_times, rv_data, rv_uncs,
    test_params, prior_unc):
    with pm.Model() as model:
        period = pm.Normal('period', mu=test_params['period'], sigma=prior_unc['period'], shape=num_planets)
        epoch = pm.Normal('t0', mu=test_params['t0'], sigma=prior_unc['t0'], shape=num_planets)
        rv_semiamp = pm.Normal('K', mu=test_params['K'], sigma=prior_unc['K'], shape=num_planets)
        
        ecc, omega_vec = _add_fixed_eccentricity(ecc_flag)

        orbit = xo.orbits.KeplerianOrbit(
            period=period,
            t0=epoch,
            # b=impact_param,
            ecc=ecc,
            cos_omega=omega_vec[0],
            sin_omega=omega_vec[1],
            # r_star=radius_star,
        )

        period_rv_only = add_uniform_prior('period_rv_only', test_params, prior_unc, shape=num_planets_rv_only)
        rv_semiamp_rv_only = add_uniform_prior('K_rv_only', test_params, prior_unc, shape=num_planets_rv_only)
        epoch_rv_only = pm.Uniform(
            't0_rv_only', lower=test_params['t0_rv_only']-period_rv_only/2,
            upper=test_params['t0_rv_only']+period_rv_only/2,
            testval=test_params['t0_rv_only'],
            shape=num_planets_rv_only)
        orbit_rv_only = xo.orbits.KeplerianOrbit(
            period=period_rv_only,
            t0=epoch_rv_only,
        )

        num_rvs = len(rv_data)
        pm.Normal('rv_gamma', mu=test_params['rv_gamma'], sigma=prior_unc['rv_gamma'], shape=num_rvs)
        pm.HalfNormal(
            'rv_jitter', sigma=prior_unc["rv_jitter"], shape=num_rvs)
        
#         model_set_up_polynomial_detrend(model, rv_times, 2)
        
#         trend_func = trend_generator
#         if trend_func is None:
#             def trend_func(model, num_rv):
#                 for i in range(num_rv):
#                     yield 0.

        rv_residuals = []
        for i, (rv_time, rv, rv_unc) in enumerate(zip(
            rv_times, rv_data, rv_uncs)): #, trend_func(model, num_rvs))):
            rv_predict = pm.Deterministic(f'rv_pred_{i}', orbit.get_radial_velocity(rv_time, K=model.K))
            rv_only_pred = pm.Deterministic(
                f'rv_only_pred_{i}',
                orbit_rv_only.get_radial_velocity(rv_time, K=rv_semiamp_rv_only))
            rv_predict_sum = (
                tt.sum(rv_predict, axis=1)
                + (tt.sum(rv_only_pred, axis=1) if num_planets_rv_only > 1 else rv_only_pred)
                + model.rv_gamma[i] # + trend
            )

            rv_residuals.append(rv - rv_predict_sum)

            total_unc = tt.sqrt(rv_unc * rv_unc + model.rv_jitter[i] * model.rv_jitter[i])
            obs_likelihood = pm.Normal(
                f"rv_obs_{i}",
                mu=rv_predict_sum,
                sd=total_unc,
                observed=rv,
            )

        # model_set_up_gp_sho(model, rv_times, rv_residuals, rv_uncs, model.rv_jitter)

        pm.Deterministic('log_prob', model.logpt)
    return model

In [ ]:
rv_model = make_rv_model_with_rv_only_planets(
    2, [False, True], 2, rv_times, rv_data, rv_uncs, test_params, prior_unc)

rv_map_soln = optimize_model(
    rv_model,
    [
        ['rv_jitter', 'rv_gamma', 'K', 'K_rv_only'],  #, 'gp_sigma', 'gp_rho', 'gp_tau'],#, 'rv_trend_coeff'],
        ['t0', 'period', 't0_rv_only', 'period_rv_only'],
        # ['gp_sigma', 'gp_rho', 'gp_tau'],
    ]
)
rv_map_soln

In [ ]:
fig, rv_folded_axs, rv_unfolded_ax, rv_residual_ax = plotting.make_multi_planet_rv_axes(4)
rv_unfolded_ax.set_ylabel('RV ($\mathrm{m}\,\mathrm{s}^{-1}$)')
rv_residual_ax.set_ylabel('Residuals ($\mathrm{m}\,\mathrm{s}^{-1}$)')
rv_residual_ax.set_xlabel('$\mathrm{BJD}_\mathrm{TDB} - 2{,}457{,}000$')
rv_folded_axs[0].set_title('c')
rv_folded_axs[1].set_title('b')
for ax in rv_folded_axs:
    ax.set_xlabel('Phase')
    ax.set_ylabel('RV ($\mathrm{m}\,\mathrm{s}^{-1}$)')

plot_orbit = xo.orbits.KeplerianOrbit(
    **{k: rv_map_soln[k] for k in ['period', 't0', 'ecc', 'omega']})

plot_orbit = xo.orbits.KeplerianOrbit(
    period=np.concatenate([rv_map_soln['period'], rv_map_soln['period_rv_only']]),
    t0=np.concatenate([rv_map_soln['t0'], rv_map_soln['t0_rv_only']]),
    ecc=np.concatenate([rv_map_soln['ecc'], [0.]*num_planet_rv_only]),
    omega=np.concatenate([rv_map_soln['omega'], [np.pi/2]*num_planet_rv_only]),
)

plot_rv_data_style = {
    'markeredgecolor': '#2a0944',
    'markeredgewidth': 0.5,
    'alpha': 0.7,
    'markersize': 3,
    'ecolor': 'gray',
    'elinewidth': 1,
}
    
plot_rv_inst_styles = [
    {'color': '#3b185f', 'fmt': '^'},
    {'color': '#a12568', 'fmt': 'd'},
    {'color': '#fec260', 'fmt': 'o'},
]

# rv_trend_func, rv_trend_unc_func = gp_trend_func_generator(
#     gp_sho, rv_map_soln, rv_times, rv_data, rv_uncs, return_unc=True)

plotting.plot_multi_planet_folded_rv(
    rv_folded_axs, rv_unfolded_ax, rv_residual_ax,
    4, plot_orbit, np.concatenate([rv_map_soln['K'], rv_map_soln['K_rv_only']]),
    rv_map_soln['rv_gamma'],
    rv_map_soln['rv_jitter'],
    rv_times,
    rv_data,
    rv_uncs,
    ['CHIRON', 'FEROS', 'HARPS'],
    # trends=trend_generator(rv_map_soln, len(rv_data)),
    # trends=gp_generator(rv_map_soln, len(rv_data)),
    # model_trend_func=rv_trend_func,
    # model_trend_unc_func=rv_trend_unc_func,
    # model_trend_func=gp_trend_func_generator(gp_sho, rv_map_soln, rv_times, rv_data, rv_uncs),
    # model_trend_func=lambda t: polynomial_design_matrix(t, rv_map_soln['rv_time_offset'], 2) @ rv_map_soln['rv_trend_coeff'],
    rv_data_style=plot_rv_data_style,
    rv_inst_styles=plot_rv_inst_styles,
)

rv_unfolded_ax.set_xlim(1905, 2375)
rv_unfolded_ax.legend(loc='lower center')

for ax, p in zip(rv_folded_axs, rv_map_soln["period"]):
    ax.text(0.05, 0.05, f'$P = {p:.6f}\,\mathrm{{d}}$', transform=ax.transAxes)


In [ ]:
rv_traces = []

In [ ]:
with rv_model:
    trace = pmx.sample(
        tune=5000,
        draws=5000,
        start=rv_map_soln,
        cores=32,
        chains=32,
        # initial_accept=0.5,
        target_accept=0.97,
        return_inferencedata=True,
        idata_kwargs={
            'log_likelihood': False,
        },
#         parameter_groups=[
#             pmx.ParameterGroup([simple_model.period, simple_model.t0]),
#             pmx.ParameterGroup([simple_model.tdur, simple_model.rp, simple_model.b]),
#             pmx.ParameterGroup([simple_model.mean_flux, simple_model.lc_jitter]),
#         ],
    )
rv_traces.append(trace)

In [ ]:
trace['posterior']

In [ ]:
trace.to_netcdf(
    '../chains/toi2000_rv_only_90d_17d.nc')
    # groups=['posterior', 'log_likelihood', 'sample_stats'])

In [ ]:
trace = az.from_netcdf('../chains/toi2000_rv_only_90d_17d.nc')
trace_3p = az.from_netcdf('../chains/toi2000_rv_only_90d.nc')

In [ ]:
display_var_names = [
    'period', 't0',
    'sqrt_ecc_vec_0',
    'K',
    'period_rv_only', 't0_rv_only', 'K_rv_only',
    'rv_gamma', 'rv_jitter',
    # 'gp_sigma', 'gp_rho', 'gp_tau',
]
az.plot_trace(trace, var_names=display_var_names)
plt.tight_layout()

In [ ]:
az.plot_trace(trace, var_names=['K'])

In [ ]:
trace_joint2p = az.from_netcdf('../chains/toi-2000_trace_2023-02-01T191240.nc')

In [ ]:
az.plot_trace(trace_joint2p, var_names=['K'])

In [ ]:
K_2p = trace_joint2p.posterior['K'][:, :, 1].stack(sample=("chain", "draw"))
K_4p = trace.posterior['K'][:, :, 1].stack(sample=("chain", "draw"))

fig, ax = plt.subplots(dpi=144)
_, bins, _ = ax.hist(K_2p, density=True, histtype='step', bins=50, label='joint 2-planet')
ax.hist(K_4p, density=True, histtype='step', bins=bins, label='RV-only 4-planet')

ax.legend()
ax.set_xlabel('$K$ (m/s)')
ax.set_ylabel('Probability density')
ax.set_title('TOI-2000 b posterior comparison')
fig.savefig('../plots/toi_2000_b_posterior_comparison_K.png', bbox_inches='tight')

In [ ]:
K_2p = trace_joint2p.posterior['K'][:, :, 0].stack(sample=("chain", "draw"))
K_4p = trace.posterior['K'][:, :, 0].stack(sample=("chain", "draw"))

fig, ax = plt.subplots(dpi=144)
_, bins, _ = ax.hist(K_2p, density=True, histtype='step', bins=50, label='joint 2-planet')
ax.hist(K_4p, density=True, histtype='step', bins=bins, label='RV-only 4-planet')

ax.legend()
ax.set_xlabel('$K$ (m/s)')
ax.set_ylabel('Probability density')
ax.set_title('TOI-2000 c posterior comparison')
fig.savefig('../plots/toi_2000_c_posterior_comparison_K.png', bbox_inches='tight')

In [ ]:
display_var_names = [
    'period', 't0',
    'sqrt_ecc_vec_0', 'ecc', 'omega',
    'K',
    'period_rv_only', 't0_rv_only', 'K_rv_only',
    'rv_gamma', 'rv_jitter',
    # 'gp_sigma', 'gp_rho', 'gp_tau',
]
summary_3p = az.summary(
    trace_3p, var_names=display_var_names,
    round_to=7,
    hdi_prob=0.997,
    skipna=True,
    # kind='stats',
    # stat_funcs={
    #     median
    # }
    coords={"ecc_dim_0": [0], "omega_dim_0": [0]},
    circ_var_names={'omega'},
    stat_funcs={
        'median': np.median,
        '-': lambda x: np.quantile(x, 0.16) - np.median(x),
        '+': lambda x: np.quantile(x, 0.84) - np.median(x),
    },
)
summary_3p

In [ ]:
display_var_names = [
    'period', 't0',
    'sqrt_ecc_vec_0', 'ecc', 'omega',
    'K',
    'period_rv_only', 't0_rv_only', 'K_rv_only',
    'rv_gamma', 'rv_jitter',
    # 'gp_sigma', 'gp_rho', 'gp_tau',
]
summary = az.summary(
    trace, var_names=display_var_names,
    round_to=7,
    hdi_prob=0.997,
    skipna=True,
    # kind='stats',
    # stat_funcs={
    #     median
    # }
    coords={"ecc_dim_0": [0], "omega_dim_0": [0]},
    circ_var_names={'omega'},
    stat_funcs={
        'median': np.median,
        '-': lambda x: np.quantile(x, 0.16) - np.median(x),
        '+': lambda x: np.quantile(x, 0.84) - np.median(x),
    },
)
summary

In [ ]:
rv_flat_samples = trace.posterior.stack(sample=("chain", "draw"))
rv_median_soln = {k:v.data for k, v in rv_flat_samples.median(dim='sample').items()}
rv_max_post_index = rv_flat_samples.log_prob.argmax(dim='sample')
rv_max_post_soln = {k:v.data for k, v in rv_flat_samples[{'sample': rv_max_post_index}].items()}

In [ ]:
astro.calculate_min_planet_mass_earth(
    np.array([22.807773, 5.948806, 15.426891, 6.332211]),
    np.array([0.063981, 0, 0, 0]),
    np.array([9.127056, 3.098302, 90.735801, 17.289302]),
    1.082411,
)

In [ ]:
astro.calculate_min_planet_mass_earth(
    np.array([23.7, 4.59, 15.426891, 6.332211]),
    np.array([0.063981, 0, 0, 0]),
    np.array([9.127055, 3.098302, 90.735801, 17.289302]),
    1.082411,
)

In [ ]:
astro.calculate_rv_semiamp(
    np.array([75.3, 10.2]),
    np.array([0.063981, 0, 0, 0]),
    np.array([9.127055, 3.09833]),
    1.082411,
)

In [ ]:
rng = np.random.default_rng()
random_size = np.array(rv_flat_samples['K']).shape
np.quantile(
    astro.calculate_min_planet_mass_earth(
        np.array(rv_flat_samples['K']),
        np.array(rv_flat_samples['ecc']),
        np.array(rv_flat_samples['period']),
        np.random.normal(1.082411, 0.06, random_size),
    ), np.array([0.16, 0.5, 0.84]), axis=1)

In [ ]:
(14.3-11)/2.6, (11-14.3)/14.3, (78.3-81.7)/4.7

In [ ]:
rng = np.random.default_rng()
random_size = np.array(rv_flat_samples['K']).shape
np.quantile(
    astro.calculate_min_planet_mass_earth(
        np.array(rv_flat_samples['K']),
        0.,
        np.array(rv_flat_samples['period']),
        np.random.normal(1.082411, 0.06, random_size),
    ), np.array([0.16, 0.5, 0.84]), axis=1)

In [ ]:
rng = np.random.default_rng()
random_size = np.array(rv_flat_samples['K']).shape
np.quantile(
    astro.calculate_min_planet_mass_earth(
        np.vstack([
            np.random.normal(23.7, 1.0, random_size[1]),
            np.random.normal(4.59, 1.0, random_size[1]),
        ]),
        0.,
        np.array(rv_flat_samples['period']),
        np.random.normal(1.082411, 0.06, random_size),
    ), np.array([0.16, 0.5, 0.84]), axis=1)

In [ ]:
rng = np.random.default_rng()
random_size = np.array(rv_flat_samples['K_rv_only']).shape
np.quantile(
    astro.calculate_min_planet_mass_earth(
        np.array(rv_flat_samples['K_rv_only']),
        0.,
        np.array(rv_flat_samples['period_rv_only']),
        np.random.normal(1.082411, 0.06, random_size),
    ), np.array([0.16, 0.5, 0.84]), axis=1)

In [ ]:
104.30264212 - 114.13412687, 124.37240489 - 114.13412687

In [ ]:
31.77303186-26.93358525, 22.11476994-26.93358525

In [ ]:
np.quantile(
    (astro.calculate_min_planet_mass_earth(
        np.array(rv_flat_samples['K']),
        np.array(rv_flat_samples['ecc']),
        np.array(rv_flat_samples['period']),
        np.random.normal(1.082411, 0.06, random_size),
    ) /
    np.sin(np.vstack([
        np.random.normal(87.94, 0.13, random_size[1]),
        np.random.normal(84.73, 0.5, random_size[1])])/180*np.pi)),
    np.array([0.16, 0.5, 0.84]), axis=1)

In [ ]:
np.array([73.76581984, 83.03444882]) - 78.33842635, np.array([11.80104626, 16.90607814]) - 14.33273728

In [ ]:
def rv_bic(rv_soln, rv_vals, rv_uncs, num_params):
    num_points = 0
    log_likelihood = 0
    for i, (my_rv, my_unc, my_jitter) in enumerate(zip(rv_vals, rv_uncs, rv_soln['rv_jitter'])):
        num_points += len(my_rv)
        log_likelihood += priors.log_prob_gaussian(
            rv_soln[f'rv_pred_{i}'],# + rv_soln[f'rv_gp_pred_{i}'],
            my_rv,
            my_unc*my_unc + my_jitter*my_jitter)
    return num_params * np.log(num_points) - 2 * log_likelihood

In [ ]:
rv_bic(rv_max_post_soln, rv_data, rv_uncs, 19)

In [ ]:
rv_bic(rv_max_post_soln, rv_data, rv_uncs, 16)

In [ ]:
600.0033015661337-513.21787135048135

In [ ]:
rv_max_post_soln

In [ ]:
rv_flat_samples.quantile([1-0.999999426696856, 1-0.999993204653751, 1-0.999936657516334], dim='sample')

In [ ]:
rv_max_post_soln['period_rv_only']

In [ ]:
fig, rv_folded_axs, rv_unfolded_ax, rv_residual_ax = make_multi_planet_rv_axes(2)
rv_unfolded_ax.set_ylabel('RV ($\mathrm{m}\,\mathrm{s}^{-1}$)')
rv_residual_ax.set_ylabel('Residuals ($\mathrm{m}\,\mathrm{s}^{-1}$)')
rv_residual_ax.set_xlabel('$\mathrm{BJD}_\mathrm{TDB} - 2{,}457{,}000$')
rv_folded_axs[0].set_title('b')
rv_folded_axs[1].set_title('c')
for ax in rv_folded_axs:
    ax.set_xlabel('Phase')
    ax.set_ylabel('RV ($\mathrm{m}\,\mathrm{s}^{-1}$)')

plot_orbit = xo.orbits.KeplerianOrbit(
    **{k: rv_max_post_soln[k] for k in ['period', 't0', 'ecc', 'omega']})

plot_rv_data_style = {
    'markeredgecolor': '#2a0944',
    'markeredgewidth': 0.5,
    'alpha': 0.7,
    'markersize': 3,
    'ecolor': 'gray',
    'elinewidth': 1,
}
    
plot_rv_inst_styles = [
    {'color': '#3b185f', 'fmt': '^'},
    {'color': '#a12568', 'fmt': 'd'},
    {'color': '#fec260', 'fmt': 'o'},
]

rv_trend_func, rv_trend_unc_func = gp_trend_func_generator(
    gp_sho, rv_max_post_soln, rv_times, rv_data, rv_uncs, return_unc=True)

plot_multi_planet_folded_rv(
    rv_folded_axs[::-1], rv_unfolded_ax, rv_residual_ax,
    2, plot_orbit, rv_max_post_soln['K'],
    rv_max_post_soln['rv_gamma'],
    rv_max_post_soln['rv_jitter'],
    rv_times,
    rv_data,
    rv_uncs,
    ['CHIRON', 'FEROS', 'HARPS'],
    # trends=trend_generator(rv_max_post_soln, len(rv_data)),
    trends=gp_generator(rv_max_post_soln, len(rv_data)),
    model_trend_func=rv_trend_func,
    model_trend_unc_func=rv_trend_unc_func,
    # model_trend_func=gp_trend_func_generator(rv_max_post_soln, rv_times, rv_data, rv_uncs),
    # model_trend_func=lambda t: polynomial_design_matrix(t, rv_max_post_soln['rv_time_offset'], 2) @ rv_max_post_soln['rv_trend_coeff'],
    rv_data_style=plot_rv_data_style,
    rv_inst_styles=plot_rv_inst_styles,
)

rv_unfolded_ax.set_xlim(1905, 2375)
rv_unfolded_ax.legend(loc='lower center')

for ax, p in zip(rv_folded_axs[::-1], rv_max_post_soln["period"]):
    ax.text(0.05, 0.05, f'$P = {p:.6f}\,\mathrm{{d}}$', transform=ax.transAxes)

fig.savefig('plot/toi2000_rv_only_gp_sho.pdf', bbox_inches='tight')

In [ ]:
display_var_names = [
    'period', 't0', 'sqrt_ecc_vec_0', 'K',
    'rv_gamma', 'rv_jitter',
    # 'gp_sigma', 'gp_rho',
]
_ = corner.corner(
    trace,
    quantiles=(0.16, 0.5, 0.84),
    levels=1-np.exp(-np.array([0.5, 1, 1.5, 2, 2.5])**2/2),
    var_names=display_var_names,
)